In [1]:
from pathlib import Path
import pickle
import json
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
from ase.units import create_units

from uncertainty_virial_stress_runmd import run_md_one_latparam

%matplotlib inline
plt.style.use("default")

u = create_units("2018")

In [2]:
# Read setting file
WORK_DIR = Path().absolute()
ROOT_DIR = WORK_DIR.parent
with open(ROOT_DIR / "settings.json", "r") as f:
    settings = json.load(f)
partition = settings["partition"]
RES_DIR = WORK_DIR / "results" / f"{partition}_partition"
if not RES_DIR.exists():
    RES_DIR.mkdir(parents=True)

In [3]:
alist = np.linspace(0.95, 1.05, 11) * 2.466

In [4]:
stress_ens = np.zeros((100, len(alist), 6))

for ii in tqdm(range(100)):
    path = RES_DIR / f"{ii:03d}" / "virial_stress"
    modelname = f"DUNN_losstraj_{ii:03d}"
    stress_ens[ii] = np.array([run_md_one_latparam(a, modelname, path) for a in alist])

# Convert the stress data unit into GPa
stress_ens = stress_ens * u["bar"] / u["GPa"]

100%|███████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.58it/s]


In [5]:
stress_mean = np.mean(stress_ens, axis=0)
stress_err = np.std(stress_ens, axis=0)

In [6]:
# Export the data needed to plot the result
plot_data_dict = {
    "stress": {"values": stress_ens, "metadata": "Virial stress in GPa"},
    "alist": {"value": alist, "metadata": "List of lattice parameters in angstrom"},
}
with open(RES_DIR / "uncertainty_virial_stress.pkl", "wb") as f:
    pickle.dump(plot_data_dict, f, protocol=4)

In [7]:
plt.figure()
plt.errorbar(
    alist, -stress_mean[:, 0], stress_err[:, 0], capsize=3, label=r"$\sigma_{11}$"
)
plt.errorbar(
    alist, -stress_mean[:, 1], stress_err[:, 1], capsize=3, label=r"$\sigma_{22}$"
)
plt.xlabel(r"Lattice parameter $\AA$")
plt.ylabel("Virial stress (GPa)")
plt.ylim(-65, 65)
plt.legend()
plt.show()